# Crimson Responses - kilosort

basically the same thing as the other crimson response stuff, except we're working with kilosort stuff

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.decomposition import PCA
import os, glob, re
import openephys_utils 
from tqdm.notebook import tqdm
import kilosort

%matplotlib qt


Get firing rates from kilosort data -- since this will need to work differently from the version I did with the "spike dictionary"

In [ ]:
def kilosort_FR(kilosort_dir:str, bin_sec:float = .01, fs:int = 30000):
    '''
    Return the firing rates for each template

    inputs:
        kilosort_dir:str        - directory where the kilosort results were stored
        bin_sec:float           - bin length in seconds [.01]
        fs:int                  - sampling rate [30000]

    outputs:
        firing_rates:np.array   - TxN array of firing rates of N templates
    '''
    spike_times = np.load(os.path.join(kilosort_dir, 'spike_times.npy'))
    spike_templates = np.load(os.path.join(kilosort_dir, 'spike_templates.npy'))

    bin_times = np.arange(np.ceil(spike_times[-1]/fs), step=bin_sec)
    firing_rates = np.ndarray((spike_templates.unique().shape[0], bin_times.shape[0]))
    for template in np.unique(spike_templates):
        firing_rates[:,template] = np.hist(spike_times[spike_templates == template], bins=bin_times)

    return firing_rates, bin_times[:-1]




    

In [ ]:
# np.arange(np.ceil(spike_times[-1]/(30000)), step=.01)
aa = np.arange(np.ceil(spike_times[-1]/(30000*.01)))
bb = np.arange(np.ceil(spike_times[-1]/(30000)), step=.01)
cc = np.arange(np.ceil(spike_times[-1]), step=(30000*.01))

print(f'aa:{aa.shape}, bb:{bb.shape}, cc:{cc.shape}')
print(f'aa:{aa[-1]}, bb:{bb[-1]}, cc:{cc[-1]}')

batch run through kilosort on filtered data

october 2nd

In [ ]:
# get a list of directories
base_dir = 'Z:\\BrainPatch\\20241002\\lateral\\'
directories = [directory for directory in os.listdir(base_dir) if '2ms' in directory]

# probe and settings
probe_name = 'Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat'
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

# sos for a BPF
filtered = True # should we filter the data?
sos = signal.butter(N = 8, Wn = [300, 6000], btype='bandpass', fs=30000, output='sos') if filtered else None

# for directory in tqdm(directories):
for i_directory,directory in enumerate(directories):
    if i_directory > 0:
        continue

    # pull out the current and distance
    current = int(re.search('(\d{1,2})mA', directory)[1])
    distance = int(re.search('(\d{3,4})um', directory)[1])

    # print it all out
    print(f'{current}mA {distance}mm')

    # get sig_eraasr if it doesn't exist already
    if not os.path.exists(os.path.join(directory, 'sig_eraasr.npy')):
        sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(os.path.join(base_dir,directory))

        # clean, pull out threshold crossings, get firing rates
        sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
        # spike_dict = openephys_utils.threshold_crossings(sig_eraasr, multi_rejection=None, low_cutoff=-20)
        # fr, bins = openephys_utils.calc_FR(spike_dict, max_samp = sig_eraasr.shape[0], min_samp = 0, bin_sec = bin_sec)
    else:
        sig_eraasr = np.load(os.path.join(directory, 'sig_eraasr.npy'))

    # filter at 300, 6000
    if filtered:
        sig_filter = signal.sosfiltfilt(sos, sig_eraasr, axis=0)
        np.save(os.path.join(directory, 'sig_filter.npy'))


    # kilosort
    if not filtered:
        settings['filename'] = os.path.join(directory,'sig_eraasr.npy')
        print('kilosort on unfiltered')
        results_dir=os.path.join(directory, 'kilosort4_unfiltered')
        kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=results_dir)
    else:
        settings['filename'] = os.path.join(directory,'sig_filter.npy')
        print('kilosort on filtered')
        results_dir=os.path.join(directory, 'kilosort4_filtered')
        kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=results_dir)

    # firing rates from kilosort
    firing_rate, bins = kilosort_FR(results_dir)
    



In [ ]:
spike_times = np.load('Z:\\BrainPatch\\20241002\\lateral\\Crimson__2024-10-02_12-21-01__20mA_2ms_400um\\kilosort4_filter\\spike_times.npy')
spike_templates = np.load('Z:\\BrainPatch\\20241002\\lateral\\Crimson__2024-10-02_12-21-01__20mA_2ms_400um\\kilosort4_filter\\spike_templates.npy')